In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
import plotly.graph_objects as go
import random

from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

In [128]:
data = pd.read_csv('Median_Income_and_AMI_(census_tract).csv')

data.isna().sum()

data.drop(data.loc[data['med_hh_income'].isna()].index, inplace=True)

In [129]:
data.csa.unique()

array(['Los Angeles - Wholesale District',
       'Los Angeles - Lincoln Heights', 'Los Angeles - El Sereno',
       'Los Angeles - Highland Park', 'Los Angeles - University Hills',
       'Los Angeles - Boyle Heights', 'City of Pomona',
       'City of Diamond Bar', 'Unincorporated - Rowland Heights',
       'City of Industry', 'City of Walnut', 'Unincorporated - Covina',
       'City of Covina', 'City of San Dimas',
       'Unincorporated - Covina (Charter Oak)', 'City of Glendora',
       'Unincorporated - Azusa', 'City of Azusa', 'City of Norwalk',
       'City of Artesia', 'City of Lakewood', 'City of Hawaiian Gardens',
       'City of Long Beach', 'City of Cerritos',
       'Los Angeles - Little Tokyo', 'Los Angeles - Chinatown',
       'Los Angeles - Downtown', 'Los Angeles - Temple-Beaudry',
       'Los Angeles - Historic Filipinotown', 'Los Angeles - Westlake',
       'City of Irwindale', 'City of Baldwin Park', 'City of West Covina',
       'City of Signal Hill', 'Los Angeles

In [130]:
area = pd.read_csv('zip.csv').drop('Unnamed: 0', axis=1)

filtered_data = pd.DataFrame()

for x in area['AREA NAME']:
    df=data.loc[data['csa'].str.contains(x)]
    filtered_data = pd.concat([filtered_data, df])


filtered_data['csa'].unique()

array(['Los Angeles - Central', 'Los Angeles - Vernon Central',
       'Los Angeles - Mission Hills',
       'Unincorporated - Northeast San Gabriel',
       'Los Angeles - Harbor City', 'Los Angeles - Harbor Gateway',
       'Los Angeles - Harbor Pines', 'Los Angeles - Van Nuys',
       'Unincorporated - West LA', 'Los Angeles - Wilshire Center',
       'Los Angeles - Pacific Palisades',
       'Unincorporated - Southeast Antelope Valley',
       'Los Angeles - North Hollywood', 'City of West Hollywood',
       'Los Angeles - Hollywood Hills', 'Los Angeles - Hollywood',
       'Los Angeles - East Hollywood'], dtype=object)

In [131]:
dfm = pd.concat(
    [filtered_data,
    data.loc[data['csa'] == 'Los Angeles - Woodland Hills'],
    data.loc[data['csa'] == 'Los Angeles - Sunland'],
    data.loc[data['csa'] == 'Los Angeles - Canoga Park'],
    data.loc[data['csa'] == 'Los Angeles - San Pedro'],
    data.loc[data['csa'] == 'Los Angeles - Mar Vista']
    ])

dfm['city']=dfm.csa.str.split('-', expand=True)[1]
dfm = dfm.loc[:, ['city', 'med_hh_income']]    
dfm = dfm.groupby('city')['med_hh_income'].mean().reset_index()



In [132]:
dfm.loc[0, 'city'] = 'West Valley'
dfm.loc[2, 'city'] = 'E Hollywood'
dfm.loc[3, 'city'] = 'Harbor'
dfm.loc[4, 'city'] = 'Harbor'
dfm.loc[5, 'city'] = 'Harbor'
dfm.loc[8, 'city'] = 'Pacific'
dfm.loc[9, 'city'] = 'Mission'
dfm.loc[10, 'city'] = 'N Hollywood'
dfm.loc[15, 'city'] = 'Foothill'
dfm.loc[19, 'city'] = 'Wilshire'
dfm.loc[20, 'city'] = 'Topanga'

In [133]:
dfm = dfm.drop([7, 11, 12, 14, 17, 2, 10, 13]).groupby('city')['med_hh_income'].mean().reset_index()

In [134]:
dfm.city = dfm.city.str.replace(' ', '')
dfm.rename(columns={'city':'area_name'}, inplace=True)

In [135]:
crime_cnt = pd.read_csv('crim cnt in 2020_2021.csv').drop(['Unnamed: 0'], axis=1)

In [138]:
df = pd.merge(crime_cnt, dfm, on='area_name')
df

,area_name,crime_2020,crime_2021,med_hh_income
0,Central,11586,13145,45796.777778
1,Wilshire,9272,9716,46770.666667
2,Hollywood,10140,12077,68935.304348
3,Pacific,11556,12907,90890.250000
4,Harbor,8856,8853,78880.513889
5,Foothill,7093,7023,93109.833333
6,Mission,8458,8208,76706.000000
7,Topanga,8091,8566,110807.000000


In [139]:
df.corr()

,crime_2020,crime_2021,med_hh_income
crime_2020,1.000000,0.978228,-0.482280
crime_2021,0.978228,1.000000,-0.444948
med_hh_income,-0.482280,-0.444948,1.000000


In [159]:
px.scatter(df, 
            y=['crime_2020', 'crime_2021'],
            x='med_hh_income',
              trendline='ols', 
          size='med_hh_income',
          title='Crime Cnt 2020/2021 vs. Median Household Income')\
.update_layout(template=dict(layout=dict(plot_bgcolor='#f5e6ca')))

In [149]:
crime_rate = pd.read_csv('crime_rate(2020_2021).csv').drop(['Unnamed: 0'], axis=1)

In [154]:
dff = pd.merge(crime_rate, dfm, on='area_name')
dff

,area_name,crime_rate_2020,crime_rate_2021,med_hh_income
0,Mission,0.447632,0.437713,76706.000000
1,Foothill,0.328562,0.329889,93109.833333
2,Hollywood,0.313641,0.404089,68935.304348
3,Wilshire,0.244084,0.263235,46770.666667
4,Central,0.217987,0.241707,45796.777778
5,Pacific,0.208954,0.235045,90890.250000
6,Topanga,0.174007,0.190648,110807.000000
7,Harbor,0.144541,0.141316,78880.513889


In [155]:
dff.corr()

,crime_rate_2020,crime_rate_2021,med_hh_income
crime_rate_2020,1.000000,0.952768,-0.091619
crime_rate_2021,0.952768,1.000000,-0.151178
med_hh_income,-0.091619,-0.151178,1.000000


In [156]:
px.scatter(dff, 
            y=['crime_rate_2020', 'crime_rate_2021'],
            x='med_hh_income',
              trendline='ols', 
          size='med_hh_income',
          title='Crime Cnt 2020/2021 vs. Median Household Income')